# **Ф36**: Модуль генерации ценностно-ориентированных требований к трансформации территорий

## 1. Читаем подготовленные данные

In [5]:
import pandas as pd
import geopandas as gpd
from blocksnet.config import log_config

log_config.set_disable_tqdm(True)
log_config.set_logger_level('ERROR')

service_types = pd.read_pickle('./data/service_types.pickle')
blocks = pd.read_pickle('./data/blocks.pickle')
acc_mx = pd.read_pickle('./data/acc_mx.pickle')

Берем лишь те типы сервисов, где известны ценности, которые они поддерживают

In [6]:
service_types = service_types[~service_types['social_values'].isna()].copy()

## 2. Вспомогательный код и оценка

1. Считаем обеспеченности по каждому типу сервиса (если столбца нет, то у нас и типа сервиса нет по идее вокруг и на территории).
2. Обрезаем до уровня контекста.
3. Считаем суммарную оценку по контексту кварталов и ассоциируем её с ценностью.

P.S: под контекстом в данном случае понимаем не контекст в рамках Простора, а контекст в рамках доступности квартала.

In [16]:
from blocksnet.relations.accessibility import get_accessibility_context

def _get_project_context(blocks : pd.DataFrame, acc_mx : pd.DataFrame, accessibility : float) -> list[int]:
  project_blocks = blocks[~blocks.is_context].copy()
  context_blocks = get_accessibility_context(acc_mx, project_blocks, accessibility, out=False)
  return list(context_blocks.index)

In [30]:
from blocksnet.analysis.provision import competitive_provision, provision_strong_total
from blocksnet.config import service_types_config

def _assess_provision(blocks : pd.DataFrame, acc_mx : pd.DataFrame, service_type : str):
  _, demand, accessibility = service_types_config[service_type].values()
  capacity_column = f'capacity_{service_type}'
  if capacity_column not in blocks.columns:
    return 0.0
  context_ids = _get_project_context(blocks, acc_mx, accessibility)
  blocks_df = blocks.rename(columns={capacity_column:'capacity'})[['population', 'capacity']].copy()
  prov_df, _ = competitive_provision(blocks_df, acc_mx, accessibility, demand)
  prov_df = prov_df.loc[context_ids].copy()
  if prov_df['demand'].sum() == 0:
    return None
  return float(provision_strong_total(prov_df))

In [31]:
from tqdm import tqdm

social_values_provisions = {}

for st_id in tqdm(service_types.index):
  st_name = service_types.loc[st_id,'name']
  social_values = service_types.loc[st_id,'social_values']
  prov_total = _assess_provision(blocks, acc_mx, st_name)
  for social_value in social_values:
    if social_value in social_values_provisions:
      social_values_provisions[social_value].append(prov_total)
    else:
      social_values_provisions[social_value] = [prov_total]

100%|██████████| 37/37 [00:02<00:00, 16.51it/s]


## 2. Результат

soc_value_id -> уровень поддержки ценности

In [39]:
import numpy as np

def _get_value_level(provisions : list[float | None]) -> float:
  provisions = [p for p in provisions if p is not None]
  return np.mean(provisions)

index = social_values_provisions.keys()
columns = ['social_value_level']
result_df = pd.DataFrame(data=[_get_value_level(social_values_provisions[sv_id]) for sv_id in index], index=index, columns=columns)
result_df

,social_value_level
8,0.746403
2,0.664122
4,0.332061
10,0.000000
17,0.333333
15,0.642689
16,0.676887
11,0.661871
3,0.985612
7,0.263034
